In [1]:
import pandas as pd
import time as time
import requests

In [2]:
# This function scrapes tot_posts from Reddit including [removed] and [deleted] posts

def scrape_reddit(subreddit, tot_posts):
    
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()
    posts_scraped = 0

    while len(df) < tot_posts:
        if tot_posts < 100:
            params = {
                'subreddit': subreddit,
                'size': tot_posts,
                'before': int(time.time())
            }
            res = requests.get(url, params)
            data = res.json()
            posts = data['data']
            df = pd.DataFrame(posts)
        else:
            if len(df) == 0:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': int(time.time())
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df = pd.DataFrame(posts)
                posts_scraped += 100
            elif tot_posts - posts_scraped > 100:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
                posts_scraped += 100
            else:
                params = {
                    'subreddit': subreddit,
                    'size': (tot_posts - posts_scraped),
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
        
    return df

In [3]:
ask_men = scrape_reddit('AskMen', 1000)

In [4]:
len(ask_men)

1000

In [5]:
ask_men[['subreddit', 'id', 'title', 'selftext', 'domain']].head(25)

,subreddit,id,title,selftext,domain
0,AskMen,j9k89r,"When girls get attracted to you with you, is i...",Seriously I've noticed whenever I'm in a high ...,self.AskMen
1,AskMen,j9k60k,How can I tell if a guy likes getting blow jobs?,[removed],self.AskMen
2,AskMen,j9k4zd,I’m having a slight fear of infertility despit...,[removed],self.AskMen
3,AskMen,j9k49w,What's the dumbest thing love made you do?,,self.AskMen
4,AskMen,j9k35h,How does your texting style change when you're...,,self.AskMen
5,AskMen,j9jxgb,What are some podcasts you find useful?,[removed],self.AskMen
6,AskMen,j9jszp,How do you deal with a bad haircut?,The barber just completely ended me and I don’...,self.AskMen
7,AskMen,j9jsz1,What's your reaction when you finally see a fe...,,self.AskMen
8,AskMen,j9jrkn,What age do balls stop growing?,[removed],self.AskMen
9,AskMen,j9jr6f,"men of reddit, what is your opinion on feminism?",[removed],self.AskMen


In [6]:
# https://stackoverflow.com/questions/20995196/python-pandas-counting-and-summing-specific-conditions

# ask_men[ask_men.selftext != '[removed]'].sum()   

len(ask_men[(ask_men.selftext != '[removed]') & (ask_men.selftext != '[deleted]')])

440

In [7]:
# This function scrapes at least min_posts from Reddit not including [removed] and [deleted] posts

def scrape_reddit(subreddit, min_posts):
     
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()
    posts_scraped = 0

    while len(df) < min_posts:
#         print('len =', len(df))
        if len(df) == 0:
            params = {
                'subreddit': subreddit,
                'size': 100,
                'before': int(time.time())
            }
            res = requests.get(url, params)
            data = res.json()
            posts = data['data']
            df = pd.DataFrame(posts)
            df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
            df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            
        else:
            params = {
                'subreddit': subreddit,
                'size': 100,
                'before': df.iloc[-1]['created_utc']
            }
            res = requests.get(url, params)
            data = res.json()
            posts = data['data']
            df2 = pd.DataFrame(posts)
            df = df.append(df2, ignore_index=True)
            df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
            df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
        
    return df

In [8]:
ask_men = scrape_reddit('AskMen', 300)

In [9]:
ask_men.shape

(317, 64)

In [10]:
ask_men[['subreddit', 'id', 'title', 'selftext', 'domain']].head(25)

,subreddit,id,title,selftext,domain
0,AskMen,j9k8qy,"Men of reddit, when girls get attracted to you...",Seriously I've noticed whenever I'm in a high ...,self.AskMen
1,AskMen,j9k89r,"When girls get attracted to you with you, is i...",Seriously I've noticed whenever I'm in a high ...,self.AskMen
2,AskMen,j9k49w,What's the dumbest thing love made you do?,,self.AskMen
3,AskMen,j9k35h,How does your texting style change when you're...,,self.AskMen
4,AskMen,j9jszp,How do you deal with a bad haircut?,The barber just completely ended me and I don’...,self.AskMen
5,AskMen,j9jsz1,What's your reaction when you finally see a fe...,,self.AskMen
6,AskMen,j9jr1t,How do I get some direction in my life at 23?,So I am going to be 23 this year and I have be...,self.AskMen
7,AskMen,j9jo4o,How important is respect and are you willing t...,,self.AskMen
8,AskMen,j9jnfl,How do you overcome body insecurity?,Not a day goes by where I don't cringe at my s...,self.AskMen
9,AskMen,j9jnbq,"How important is respect, how do you earn it, ...",,self.AskMen


In [23]:
# This function scrapes at least min_posts from Reddit not including [removed] and [deleted] posts, and whether a post is_self or not

def scrape_reddit(subreddit, min_posts, is_self):
     
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()
    posts_scraped = 0

    while len(df) < min_posts:
        print('len =', len(df))
        if is_self == 'Y':
            if len(df) == 0:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': int(time.time()),
                    'is_self': True
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df = pd.DataFrame(posts)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            else:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': df.iloc[-1]['created_utc'],
                    'is_self': True
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)        
        elif is_self == 'N':
            if len(df) == 0:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': int(time.time())
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df = pd.DataFrame(posts)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            else:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
                df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
        else:
            print(f"Enter 'Y' or 'N' for third argument.")
            break
        
    return df

In [24]:
intelligence = scrape_reddit('Intelligence', 300, 'U')

len = 0
Enter 'Y' or 'N' for third argument.


In [21]:
intelligence[['subreddit', 'id', 'title', 'selftext', 'domain']].head(25)

,subreddit,id,title,selftext,domain
0,Intelligence,iwqizr,Which is better,[removed]\n\n[View Poll](https://www.reddit.co...,self.Intelligence
1,Intelligence,iun1w4,Question about psyops [Chima-India conflagration],[removed]\n\n[View Poll](https://www.reddit.co...,self.Intelligence
2,Intelligence,iun1vw,Question about psyops [Chima-India conflagration],[removed]\n\n[View Poll](https://www.reddit.co...,self.Intelligence
3,Intelligence,iu9h0t,What is the most underrated Intelligence Disci...,[removed]\n\n[View Poll](https://www.reddit.co...,self.Intelligence
4,Intelligence,irqw5x,"sudden text to scott bomboy, a random politici...",NaN,self.Intelligence
5,Intelligence,hx7osk,Military Intelligence vs. Civilian Intelligenc...,NaN,self.Intelligence
6,Intelligence,h0bi1p,In-ear headphones in a SCIF,NaN,self.Intelligence
7,Intelligence,h00232,Best Eavesdropping Device in market?,NaN,self.Intelligence
8,Intelligence,gnhuwu,Military vs Graduate Route,"Besides where experience comes from, what are ...",self.Intelligence
9,Intelligence,gn7br7,What would be the hardest country to be a spy ...,I'm thinking North Korea given the whole no in...,self.Intelligence
